In [102]:
import pandas as pd
from datetime import datetime

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score


### TO DO:
- Analyse of NaN with distribution plots
- Correlation Matrix to see which features are important
- 

Retours :

- Covariate shift : AUC, s’il arrive a distinguer qu’une feature appartient plus à du train que du test (70%) on supprime la feature.
- Poids de similarité : si une observation du train est proche d’une observation du test —> pondérer
- Ensemble learning : predict_proba, mettre un seuil sur la probabilité !!
- Faire du sample_weights 

In [2]:
## Read csvs
train_df = pd.read_csv('train.csv', index_col=0)
train_x = train_df[['date', 'org', 'tld', 'ccs', 'bcced', 'mail_type', 'images', 'urls',
       'salutations', 'designation', 'chars_in_subject', 'chars_in_body',]]
train_y = train_df[['label']]
test_x = pd.read_csv('test.csv', index_col=0)

In [91]:
print('Size of the training set ', train_df.shape)
print('Size of the testing set ', test_x.shape)

Size of the training set  (24840, 13)
Size of the testing set  (10647, 12)


In [85]:
## Handle missing values with median
train_x.fillna('median', inplace=True)
test_x.fillna('median', inplace=True)

In [86]:
def date_to_datetime_format(date):
    str_days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    first_int = str(list(range(0,10)))
    date = date.strip() #remove whitespaces around the hole date
    
    #Thu, 12 May 2016 18:54:03 +0530 or Thu, 07 Jul 2016 01:11:41 -0600
    if (date[0:3] in str_days) & ((date[-5] == '+') | (date[-5] == '-')):
        #case of wrong date Mon, 6 Apr 2015 22:49:45 +0580
        if int(date[-2:]) > 60:
            date = date[:-2] + '00'
        return datetime.strptime(date.strip(), '%a, %d %b %Y %H:%M:%S %z')
    
    #Wed, 01 May 2019 09:09:26 +0000 (UTC) or 'Tue, 23 May 2017 00:20:01 -0500 (CDT)'
    elif (date[0:3] in str_days) & (date[-1] == ')') & (date[-5] == '('):
        return datetime.strptime(date[0:-6], '%a, %d %b %Y %H:%M:%S %z')
    
    #27 Nov 2017 09:20:43 +0100 or 21 Aug 2017 09:36:56 -0400 
    elif (date[0] in first_int) & ((date[-5] == '+') | (date[-5] == '-')):
        return datetime.strptime(date, '%d %b %Y %H:%M:%S %z')
    
    #Thu, 31 Mar 2016 15:32:47 +0200 (CEST) or Wed, 25 Sep 2013 13:01:35 +0300 (EEST)
    elif (date[0:3] in str_days) & (date[-6:] in ['(CEST)', '(EEST)']):
        return datetime.strptime(date[0:-7], '%a, %d %b %Y %H:%M:%S %z')
    
    #2 Sep 2016 09:55:22 +0530
    elif (date[0] in first_int) & ((date[-5] == '+') | (date[-5] == '-')):
        return datetime.strptime(date, '%d %b %Y %H:%M:%S %z')
    
    #Wed, 23 Jan 2013 03:54:45 GMT
    elif (date[0:3] in str_days) & (date[-3:]=='GMT'):
        return datetime.strptime(date[:-4], '%a, %d %b %Y %H:%M:%S')
    
    #Thu, 20 Mar 2014 03:29:24 +0530 (GMT+05:30)
    elif (date[0:3] in str_days) & (date[-11:-6]=='(GMT+'):
        return datetime.strptime(date[:-12], '%a, %d %b %Y %H:%M:%S %z')
    
    #Thu, 31 Mar 2016 15:32:47 +0200 (INDIA)
    elif (date[0:3] in str_days) & (date[-7:] == '(INDIA)'):
        return datetime.strptime(date[0:-8], '%a, %d %b %Y %H:%M:%S %z')
    
    #11-MAR-2018 20:40:58
    elif (date[0] in first_int) & (date[-1] in first_int) & (len(date) == 20):
        return datetime.strptime(date, '%d-%b-%Y %H:%M:%S')
    
    #Mon, 8 Apr 2013 09:59:21
    else:
        return datetime.strptime(date, '%a, %d %b %Y %H:%M:%S')

In [87]:
train_x['datetime'] = train_x.date.apply(lambda x : date_to_datetime_format(x))

train_x['year'] = train_x['datetime'].apply(lambda x : x.year)
train_x['month'] = train_x['datetime'].apply(lambda x : x.month)
train_x['day'] = train_x['datetime'].apply(lambda x : x.day)
train_x['hour'] = train_x['datetime'] .apply(lambda x : x.hour)

In [105]:
# instantiate labelencoder object
le = LabelEncoder()
# apply le on categorical feature columns
train_x[['org']] = train_x[['org']].apply(lambda col: le.fit_transform(col))

In [100]:
feat_enc_org = OneHotEncoder()
feat_enc_org.fit(train_x[['org']])
train_x_featurized = feat_enc_org.transform(train_x[['org']])

In [4]:
## Do one hot encoding of categorical feature
feat_enc = OneHotEncoder()
feat_enc.fit(train_x)
train_x_featurized = feat_enc.transform(train_x)
test_x_featurized = feat_enc.transform(test_x)

## Train a simple KNN classifier using featurized data
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_x_featurized, train_y)
pred_y = neigh.predict(test_x_featurized)

/Users/antoineohleyer/miniconda3/envs/ipseite/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


In [5]:
pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("knn_sample_submission.csv", index=True, index_label='Id')